In [3]:
import heapq
import numpy as np
from copy import deepcopy
class Node:
    def __init__(self, cost_matrix, assigned_positions=None, level=0):
        self.cost_matrix = cost_matrix
        self.assigned_positions = assigned_positions if assigned_positions else []
        self.level = level
        self.lowerbound = self.calculate_lowerbound()
    def __lt__(self, other):
        return self.lowerbound < other.lowerbound
    def calculate_lowerbound(self):
        cost = 0
        # Add costs of already assigned positions
        for i, j in self.assigned_positions:
            cost += self.cost_matrix[i][j]
        
        # Create a copy of cost matrix for modification
        temp_matrix = deepcopy(self.cost_matrix)
        
        # Mark assigned positions as infinity
        for i, j in self.assigned_positions:
            for k in range(len(temp_matrix)):
                temp_matrix[i][k] = float('inf')
                temp_matrix[k][j] = float('inf')
        
        # Add minimum cost for unassigned rows
        for i in range(self.level, len(temp_matrix)):
            min_val = float('inf')
            for j in range(len(temp_matrix)):
                if temp_matrix[i][j] < min_val:
                    min_val = temp_matrix[i][j]
            if min_val != float('inf'):
                cost += min_val
        
        return cost

def solve_assignment(cost_matrix):
    n = len(cost_matrix)
    pq = []  # Priority queue
    
    # Create root node
    root = Node(cost_matrix)
    heapq.heappush(pq, root)
    
    min_cost = float('inf')
    best_assignment = None
    
    while pq:
        current = heapq.heappop(pq)
        
        if current.lowerbound >= min_cost:
            continue
        
        level = current.level
        
        # If we've assigned all jobs, check if this is the best solution
        if level == n:
            if current.lowerbound < min_cost:
                min_cost = current.lowerbound
                best_assignment = current.assigned_positions
            continue
        
        # Get list of already assigned jobs
        assigned_jobs = [j for _, j in current.assigned_positions]
        
        # Try each available job for the current person
        for job in range(n):
            # Skip if job is already assigned
            if job in assigned_jobs:
                continue
                
            # Create new assignment by adding (person, job) pair
            new_assigned = current.assigned_positions + [(level, job)]
            
            # Create new node with this assignment
            child = Node(cost_matrix, new_assigned, level + 1)
            
            # Add to queue if it might give us a better solution
            if child.lowerbound < min_cost:
                heapq.heappush(pq, child)
    
    return min_cost, best_assignment
     

if __name__ == "__main__":
    cost_matrix = [
        [9, 2, 7, 8],
        [6, 4, 3, 7],
        [5, 8, 1, 8],
        [7, 6, 9, 4]
    ]
    
    min_cost, assignment = solve_assignment(cost_matrix)
    print(f"Minimum Cost: {min_cost}")
    print("Assignments:")
    for person, job in sorted(assignment):
        print(f"Person {person} -> Job {job} (Cost: {cost_matrix[person][job]})")

Minimum Cost: 13
Assignments:
Person 0 -> Job 1 (Cost: 2)
Person 1 -> Job 0 (Cost: 6)
Person 2 -> Job 2 (Cost: 1)
Person 3 -> Job 3 (Cost: 4)
